In [ ]:
from zipfile import ZipFile
import os, requests
import pandas as pd

from azure.ai.ml import MLClient
from azure.ai.ml.entities import Data
from azure.identity import DefaultAzureCredential
from azure.ai.ml.constants import AssetTypes

In [ ]:
# Need config.json file in the same folder
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
ml_client = MLClient.from_config(credential=DefaultAzureCredential(exclude_shared_token_cache_credential=True),
                     logging_enable=True)

### Download Data

In [ ]:
os.makedirs('./data', exist_ok=True)

url = 'https://github.com/asherif844/MLOps/raw/master/data/AdventureWorks-oltp-install-script.zip'
zip_data = requests.get(url)

with open('./data/adventureworks.zip', 'wb') as f:
    f.write(zip_data.content)

with ZipFile('./data/adventureworks.zip', 'r') as fzip:
    fzip.extractall('./data/csv_data')

### Transform Data

In [ ]:
header = ['TransactionID', 'ProductID', 'ReferenceOrderID', 'ReferenceOrderLineID', 'TransactionDate', 'TransactionType', 'Quantity', 'ActualCost', 'ModifiedDate']

trans_hist_df = pd.read_csv('./data/csv_data/TransactionHistory.csv', sep='\t', names=header)

trans_hist_df['PaidAmount'] = trans_hist_df['Quantity'] * trans_hist_df['ActualCost']
trans_hist_df['TransactionDate'] = pd.to_datetime(trans_hist_df['TransactionDate'])
df = trans_hist_df[['TransactionDate', 'PaidAmount']]
df.set_index('TransactionDate',inplace=True)

df = df.resample('D').mean().interpolate()
df = df['2013-07':'2014-05']
df1 = df['2013']
df2 = df['2014']

df.to_csv('./data/mlops_forecast_data.csv', index=True, header=True)
df1.to_csv('./data/mlops_forecast_data2013.csv', index=True, header=True)
df2.to_csv('./data/mlops_forecast_data2014.csv', index=True, header=True)

### Upload data

In [ ]:
transaction_ts = "azureml://datastores/demostore/paths/mlops_timeseries2/mlops_forecast_data.csv"
transaction_ts2013 = "azureml://datastores/demostore/paths/mlops_timeseries2/mlops_forecast_data2013.csv"
transaction_ts2014 = "azureml://datastores/demostore/paths/mlops_timeseries2/mlops_forecast_data2014.csv"

In [ ]:
data = Data(
    path = "./data/mlops_forecast_data.csv",
    type = AssetTypes.URI_FILE,
    description = "time series dataset for mlops",
    name = "transaction2_ts",
    version = '1'
)

ml_client.data.create_or_update(data)

In [ ]:
data = Data(
    path = "./data/mlops_forecast_data2013.csv",
    type = AssetTypes.URI_FILE,
    description = "2013 time series dataset for mlops",
    name = "transaction2_ts2013",
    version = '1'
)

ml_client.data.create_or_update(data)

In [ ]:
data = Data(
    path = "./data/mlops_forecast_data2014.csv",
    type = AssetTypes.URI_FILE,
    description = "2014 time series dataset for mlops",
    name = "transaction2_ts2014",
    version = '1'
)

ml_client.data.create_or_update(data)